In [ ]:
import os
os.environ['HSA_OVERRIDE_GFX_VERSION'] = '10.3.0'

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import torch
from sklearn.model_selection import KFold
from sklearn.utils import gen_even_slices
from sklearn.metrics import accuracy_score
import pickle

In [ ]:
data = './data/digit-recognizer/train.csv'

In [ ]:
df = pd.read_csv(data)

In [ ]:
display(df.head(1))

#### Separate images from labels

In [ ]:
X = df.drop(columns=['label'])
y = df['label']

In [ ]:
print('labels:', y.unique())
print('labels:', y.unique().size)

### See an example image

In [ ]:
sample = X.sample()
image = sample.values.reshape((28,28))
idx = sample.index
print('digit:',y.iloc[idx].values.squeeze())
plt.imshow(image)
plt.show()

### Work with GPU

In [ ]:
device = torch.device('cuda:0')
display(torch.cuda.get_device_name(device))

### Reshape images

In [ ]:
images = X.values
labels = y.values

images = images.reshape(-1, 1, 28, 28)
labels = labels.reshape(-1, 1)

images = torch.tensor(images, dtype=torch.float32, device=device)
labels = torch.tensor(labels, dtype=torch.long, device=device)

print(f"images: {images.shape}")
print(f"labels: {labels.shape}")

### Convolutional Neural Network
Before using some out-of-the-box classifier, lets try defining our own neural network using pytorch

#### Model architecture: Convolutional neural network
- Convolutional layers
- Max pooling layers
- linear layers w/ ReLU activation function
- apply softmax to output

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class NeuralNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=(3,3))
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(3,3))
        self.pool = nn.MaxPool2d(kernel_size=(2,2))
        self.fc1 = nn.Linear(800, 1024)
        self.fc2 = nn.Linear(1024, 256)
        self.fc3 = nn.Linear(256, 10)

    def forward(self, x):
        x = self.pool(F.leaky_relu(self.conv1(x)))
        x = self.pool(F.leaky_relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def predict(self, x):
        x = self.forward(x)
        y = torch.argmax(x, -1)
        return y
    
    

## Training code

### Tune Hyperparameters

In [ ]:
# hyper parameters
learning_rate = 0.0001
epochs = 18
batch_size = 256

kf = KFold(n_splits=5)
# perform CV for 5 folds
for fold, (train_index, test_index) in enumerate(kf.split(images)):
    
    # define train data for the fold
    x_train, y_train = images[train_index], labels[train_index]
    
    # define test/validation data for the fold
    x_test, y_test = images[test_index], labels[test_index]
    
    # define the model
    model = NeuralNet().to(device)  
    
    # define the optimizer
    optimizer = torch.optim.Adam(lr=learning_rate, params=model.parameters())
    loss_fn = torch.nn.CrossEntropyLoss()
    
    train_losses = []
    val_losses = []
    accuracies = []
    for epoch in range(epochs):
        model.train()
        
        slices = gen_even_slices(x_train.shape[0],x_train.shape[0]//batch_size)
        for _, slice_ in enumerate(list(slices)):
            x_batch = x_train[slice_.start:slice_.stop]
            y_batch = y_train[slice_.start:slice_.stop]
            
            optimizer.zero_grad()

            output = model(x_batch)
                        
            loss = loss_fn(output,y_batch.long().squeeze())
            loss.backward()
                        
            optimizer.step()
        
        train_losses.append(loss.item())
        model.eval()
        with torch.no_grad():
            val_output = model(x_test)
            val_loss = loss_fn(val_output, y_test.long().squeeze())
            val_losses.append(val_loss.item())

    # Plot the training and validation losses for the current fold
    plt.figure(figsize=(6, 5))
    plt.plot([i for i in range(len(train_losses))], train_losses, label='Training Loss')
    plt.plot([i for i in range(len(val_losses))], val_losses, label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title(f'Training and Validation Loss - Fold {fold + 1}')
    plt.legend()
    plt.tight_layout()
    plt.show()
    
    model.eval()
    with torch.no_grad():
        preds = model.predict(x_test)
        accuracy = accuracy_score(y_test.cpu(),preds.cpu())
        accuracies.append(accuracy)
        print(f"fold {fold + 1} accuracy: {accuracy}")
    
print(f"Mean accuracy: {np.mean(accuracies)}")

### Train the model

In [239]:
learning_rate = 0.0001
epochs = 18
batch_size = 256

# define the model
model = NeuralNet().to(device)  

# optimizer
optimizer = torch.optim.Adam(lr=learning_rate, params=model.parameters())

# loss function
loss_fn = torch.nn.CrossEntropyLoss()

# prepare the data
X, y = images, labels

for epoch in range(epochs):
    model.train()
    slices = gen_even_slices(X.shape[0],X.shape[0]//batch_size)
    for _, slice_ in enumerate(list(slices)):
        
        # prepare batches
        x_batch = X[slice_.start:slice_.stop]
        y_batch = y[slice_.start:slice_.stop]
        
        optimizer.zero_grad()

        output = model.forward(x_batch)

        loss = loss_fn(output,y_batch.long().squeeze())
        loss.backward()

        optimizer.step()
    
    model.eval()
    print(loss.item())
                
model.eval()
torch.save(model.state_dict(), './data/weights.pkl')

0.09142445027828217
0.061685554683208466
0.05406118184328079
0.03545704856514931
0.030208753421902657
0.014063914306461811
0.022435840219259262
0.024667326360940933
0.004578050691634417
0.0030519335996359587
0.0029976926743984222
0.006983173079788685
0.002945544198155403
0.0014546489110216498
0.0013710251078009605
0.001217078068293631
0.000824407150503248
0.0004694343078881502


# Test the model

In [261]:
df_test = pd.read_csv('./data/digit-recognizer/test.csv')

In [262]:
test_images = df_test.values
test_images = test_images.reshape(-1, 1, 28, 28)
test_images.shape

(28000, 1, 28, 28)

In [263]:
X_test = torch.tensor(test_images, dtype=torch.float32, device=device)

In [264]:
y = model.predict(X_test)
y.shape

torch.Size([28000])

In [267]:
submission_df = pd.DataFrame({'ImageId': [i+1 for i in range(len(y))], 'Label': y.cpu()})
submission_df.head()

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3


In [268]:
submission_df.to_csv('./data/my_submission.csv', index=False)